<a href="https://colab.research.google.com/github/jundimuhammadalfatih/ocr/blob/main/custom_easyocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==1.12.0+cu113 torchvision==0.13.0+cu113 torchaudio==0.12.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 704.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0+cu121
    Uninstalling torch-2.3.0+cu121:
      Successfully uninstalled torch-2.3.0+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.18.0+cu121
    Uninstalling torchvision-0.18.0+cu121:
      Successfully uninstalled torchvision-0.18.0+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.3.0+cu121
    Uninstalling torchaudio-2.3.0+cu121:
      Successfully uninstalled torchaudio-2.3.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fol

In [2]:
import torch
print(torch.__version__)

1.12.0+cu113


In [3]:
import torch
if torch.cuda.is_available():
  print("CUDA is available!")
else:
  print("CUDA is not available.")

CUDA is available!


In [4]:
!git clone https://github.com/JaidedAI/EasyOCR.git

Cloning into 'EasyOCR'...
remote: Enumerating objects: 2736, done.
remote: Total 2736 (delta 0), reused 0 (delta 0), pack-reused 2736
Receiving objects: 100% (2736/2736), 157.83 MiB | 21.59 MiB/s, done.
Resolving deltas: 100% (1664/1664), done.
Updating files: 100% (313/313), done.


In [16]:
cd EasyOCR/

/content/EasyOCR


In [6]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 17.4 MB/s eta 0:00:00


In [17]:
cd trainer/all_data

/content/EasyOCR/trainer/all_data


In [18]:
!unzip /content/EasyOCR/trainer/all_data/en_train_filtered.zip

Archive:  /content/EasyOCR/trainer/all_data/en_train_filtered.zip
   creating: en_train_filtered/
  inflating: en_train_filtered/944.jpg  
  inflating: en_train_filtered/947.jpg  
  inflating: en_train_filtered/953.jpg  
  inflating: en_train_filtered/954.jpg  
  inflating: en_train_filtered/957.jpg  
  inflating: en_train_filtered/961.jpg  
  inflating: en_train_filtered/963.jpg  
  inflating: en_train_filtered/964.jpg  
  inflating: en_train_filtered/972.jpg  
  inflating: en_train_filtered/975.jpg  
  inflating: en_train_filtered/976.jpg  
  inflating: en_train_filtered/977.jpg  
  inflating: en_train_filtered/978.jpg  
  inflating: en_train_filtered/979.jpg  
  inflating: en_train_filtered/980.jpg  
  inflating: en_train_filtered/981.jpg  
  inflating: en_train_filtered/982.jpg  
  inflating: en_train_filtered/983.jpg  
  inflating: en_train_filtered/984.jpg  
  inflating: en_train_filtered/986.jpg  
  inflating: en_train_filtered/987.jpg  
  inflating: en_train_filtered/988.jpg  


In [19]:
!unzip /content/EasyOCR/trainer/all_data/en_val.zip

Archive:  /content/EasyOCR/trainer/all_data/en_val.zip
   creating: en_val/
  inflating: en_val/276.jpg          
  inflating: en_val/286.jpg          
  inflating: en_val/307.jpg          
  inflating: en_val/312.jpg          
  inflating: en_val/315.jpg          
  inflating: en_val/316.jpg          
  inflating: en_val/319.jpg          
  inflating: en_val/320.jpg          
  inflating: en_val/321.jpg          
  inflating: en_val/323.jpg          
  inflating: en_val/324.jpg          
  inflating: en_val/325.jpg          
  inflating: en_val/326.jpg          
  inflating: en_val/327.jpg          
  inflating: en_val/328.jpg          
  inflating: en_val/330.jpg          
  inflating: en_val/331.jpg          
  inflating: en_val/334.jpg          
  inflating: en_val/335.jpg          
  inflating: en_val/336.jpg          
  inflating: en_val/labels.csv       


In [20]:
cd ..

/content/EasyOCR/trainer


In [21]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

In [22]:
cudnn.benchmark = True
cudnn.deterministic = False

In [23]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [24]:
opt = get_config("config_files/en_filtered_config.yaml")
train(opt, amp=False)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data/en_train_filtered
opt.select_data: ['en_train_filtered']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data/en_train_filtered	 dataset: en_train_filtered
all_data/en_train_filtered/
sub-directory:	/.	 num samples: 30
num total samples of en_train_filtered: 30 x 1.0 (total_data_usage_ratio) = 30
num samples of en_train_filtered per batch: 16 x 1.0 (batch_ratio) = 16
--------------------------------------------------------------------------------
Total_batch_size: 16 = 16
--------------------------------------------------------------------------------
dataset_root:    all_data/en_val	 dataset: /
all_data/en_val/
sub-directory:	/.	 num samples: 20
---------------------

SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [32]:
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 16.0 MB/s eta 0:00:00


In [35]:
!cp /root/EasyOCR/user_network/custom_model.yaml /root/.EasyOCR//user_network/custom_model.yaml

In [36]:
!cp /root/EasyOCR/user_network/custom_model.py /root/.EasyOCR//user_network/custom_model.py

In [37]:
!cp /root/EasyOCR/model/custom_model.pth /root/.EasyOCR//model/custom_model.pth

In [49]:
import easyocr
reader = easyocr.Reader(['en'], recog_network='custom_model')
result = reader.readtext('/content/0.jpg',detail = 0)
print(result)

['Fukushima']
